# Arcus performance metrics

In [ ]:
from __future__ import print_function

In [ ]:
from nbtemplate import display_header, display_codetoggle, get_path
display_header('CATmerrit.ipynb', status='proposal version')

In this notebook, I show the effective area and resolution we can expect for Arcus with CAT gratings in the nominal design.

### What's in this document and what is not
This document contains mostly results of simulations, not the code itself because many of the plots below require a grid of simulations, e.g. in for different energies, and take while to run. Thus, those grids are run in a separate process and the relevant results (e.g. the position of the final rays on the detector) is saved in a fits file. In this notebook we then parse these fits files to extract the relevant information.


## What's included in the simulations?
Details of the setup for the simulations are obviously defined in the python code, but a list of the most important effects that are included is here:

- On-axis, point-like source
- Spacecraft pointing with jitter
- Mirror with in-plane and out-of-plane scatter
- two optical axes, each of them used by one pair of channels
- SPO and grating facet positions identical with the mechanical design
- facets are flat (i.e. simulation includes finite size effects)
- CCDs are flat and have pixels
- CCD gaps between individual chips.

## Caveats

No simulation is perfect and there are many details of the Arcus design that are not or only very marginally relevant for the optical performance that are not covered in these simulations. The main caveats with respect to the optical design are:

- The SPO model for a mirror is simplified. Instead of a full surface in 3d, SPOs are represented but a single reflection in a plane. Comparison with experimental data shows that this is a valid approximation for on-axis sources, but cannot reproduce the effects expected for off-axis sources (most notably, the vignetting).
- Tolerances: Effective area and resolving power depend on the accurate placement of SPOs, CAT gratings and CCDs. The simulations here are mostly done placing all these elements at their nominal position.

In [ ]:
import os
import sys
from glob import glob
import functools

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib inline

import astropy
import astropy.coordinates
from astropy.table import Table, Column
import astropy.units as u

from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)


In [ ]:
from cycler import cycler

In [ ]:
import marxs
from marxs.source import PointSource, FixedPointing
from marxs.analysis.gratings import resolvingpower_from_photonlist, weighted_per_order
import arcus
import arcus.arcus as instrum

In [ ]:
import mpld3
#mpld3.disable_notebook()

In [ ]:
r0 = Table.read(rayfiles[0])
print('Files were written with')
print('MARXS version', r0.meta['MARXSVER'])
print('ARCUS version', r0.meta['ARCUSVER'])
print('ARCUS inpudata git hash', r0.meta['ARCDATHA'])
print('NOTE THAT THESE VERSIONS CAN BE DIFFERENT FROM THE LAST TIME THE NOTEBOOK WAS RUN')


In [ ]:
res, aeff, energy, wave, aeff_4, res_4 = aeffRfromraygrid(get_path('raygrid-perfect'), instrum)

In [ ]:
mres, maeff, menergy, mwave, maeff_4, mres_4 = aeffRfromraygrid(get_path('raygrid'), instrum)

## Effective area

In [ ]:
#colorcycle = ['#348ABD', '#7A68A6', '#A60628', '#467821', '#CF4457', '#188487', '#E24A33']

def plot_aeff(aeff, plot_orders, ax=None):
    if ax is None:
        fig = plt.figure()
        ax = fig.add_subplot(111)

    for o in plot_orders: #, colorcycle):
        if o !=0:
            ax.plot(wave, aeff[:, orders == o], label='order {0}'.format(abs(o)))
        else:
            ind0 = (orders == 0)
            ax.plot(wave, aeff[:, ind0], 'k:', label='0th order')
    ax.plot(wave, np.sum(aeff[:, orders != 0], axis=1), 
            'k-', label='all dispersed\norders')

    ax.set_xlabel('wavelength [$\AA{}$]')
    ax.set_ylabel('$A_\mathrm{eff}$ [cm$^2]$')
    temp = ax.set_xlim([8, 55])
    return ax

In [ ]:
with mpl.style.context('fivethirtyeight'):
    plt.rc('axes', prop_cycle=(cycler('linestyle', ['-', '--', ':']) * 
                           cycler('color', ['r', 'b', 'y', 'c', 'g', 'm', '#7A68A6', '#188487', '#E24A33'])))
    # Do the same plot but split up by energy ranges
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111)
    plot_aeff(aeff_4, np.arange(-9, 1), ax)
    ax.set_title('Effective area summed over all 4 channels')
    fig.subplots_adjust(bottom=0.13, left=0.13, top=.85)
    ax.set_xlim([8, 75])
    ax.legend()
    fig.savefig(os.path.join(get_path('figures'), 'Aeff_16CCDs.png'))
    fig.savefig(os.path.join(get_path('figures'), 'Aeff_16CCDs.pdf'))

Effective area curve for Arcus based on the instrument setup and nominal alignment for every element. The effective area for all four channels is added. The contribution of individual grating orders is shown with colored curves. Each of the curves has a number of dips in it when an order falls into a chip gap. For any individual channel, the effective area drops to 0 in this case, but the channels are placed such the the chip gaps to no coincide in all of them, so the summed effective area suffers, but some signal is still detected.

The black curve sums the effective area of all detected dispersed orders, irrespective of their spectral resolving power. However, for most of the wavelength range, only orders that are very similar in resolving power (e.g. orders 4 and 5 at 30 $\unicode{xC5}$) contribute anyway. The only exception to this rule is the region around 12 $\unicode{xC5}$, where both order 1 and 9 contribute to the observed signal, with (as we will see below) significantly different $R$.

In [ ]:
with mpl.style.context('fivethirtyeight'):
    plt.rc('axes', prop_cycle=(cycler('linestyle', ['-', '--', ':']) * 
                           cycler('color', ['r', 'b', 'y', 'c', 'g', 'm', '#7A68A6', '#188487', '#E24A33'])))
    # Do the same plot but split up by energy ranges
    fig = plt.figure(figsize=(12, 10))
    for i in apertures:
        ax = fig.add_subplot(2, 2, i+1)
        plot_aeff(aeff[:, i, :], np.arange(-9, 1), ax)
        ax.set_title('Effective area for channel {}'.format(i))
        
    fig.subplots_adjust(bottom=0.13, left=0.13, top=.85)
    
    #fig.savefig(os.path.join(get_path('figures'), 'Aeff_16CCDs.png'))
    #fig.savefig(os.path.join(get_path('figures'), 'Aeff_16CCDs.pdf'))

This plot shows the effective area for each of the four channels separately, using the same color coding as above. The channels are very similar and at this scale the difference can hardly be seen, but in particular there is a slight offset in the position of the chip gaps between channel 0 and 1 on the one side and 2 and 3 on the other. Note that for each order, the effective area drops to 0, when the order falls into a chip gap, but the effective area summed over all orders does not, at least in those regions where more than one order contributes signal.

In [ ]:
#This is another way of showing which CCDs are detecting the most signal. As in the plot above, the CCD ID number is shown on the x-axis, but instead of summing the signal over all wavelengths, it's shown here for each wavelength. 
#There are two pairs of channels, so everything is mirrored at about CCD ID 32 or so. The figure shows roughly diagonal strips. As the wavelength increases from the top towards the bottom of the plot, a particular grating order gets dispersed to larger angles, causing the strip to move outward. As the order moves away from the preferred angle, it gets wekaer and weaker and more photons are found in the next lower order, which can be seen as a next line appearing. For most wavelength the signal is concentated in one or two orders for each wavelength.

In [ ]:
#Looking ot the two plots above, there are two separate effects to be seen:
#
#- Chip gaps: These are unavoidable, but as stated above it is possible that they appear to be absent for some of the simulations because the wavelength grid is too rough rought now. Thus, lines that seem very similar except for a few sudden drops are probably just that - very similar.
#- Large structure: The blaze peak is so wide that it is well covered by a set of 8 CCDs. Moving these by just a little (+- half a CCD) only changes the number of rays detected at the edges and that has only a very small effect of the total effective area.
#
#In summary, I can say that optimization of the chip placement on a scale smaller than one chip size (about 50 mm) do not make much of a difference at this point.

## Spectral resolving power

I define the resolving power as:
$R = \frac{\lambda}{\Delta \lambda} = \frac{d_x}{FWMH}$
where $\lambda$ is the wavelength of a spectral line with negligible intrinsic
width, and $\Delta \lambda$ is the observed width of this feature. Since the
detector does not give the wavelength directly, $d_x$ and the $FWHM$ are linear
distances measured along a curved detector that follows the Rowland circle. The $FWMH$ is the full width at
half maximum of the event distribution and $d_x$ is the distance between
the center of a diffracted order and the zeroth order.

In [ ]:
def plot_res(res, ax=None):
    if ax is None:
        fig = plt.figure()
        ax = fig.add_subplot(111)

    for i in np.arange(3, 10):
        ax.plot(wave, res[:, i], label='order {0}'.format(np.abs(orders[i]))
    ax.set_xlabel('wavelength [$\AA{}$]')
    ax.set_ylabel('resolving power')
    temp = ax.set_xlim([8, 55])
    return ax

In [ ]:
with mpl.style.context('fivethirtyeight'):
    plt.rc('axes', prop_cycle=(cycler('linestyle', ['-', '--', ':']) * 
                               cycler('color', ['r', 'b', 'y', 'c', 'g', 'm', '#7A68A6', '#188487', '#E24A33'])))

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax = plot_res(res_4, ax)
    ax.legend()
    temp = ax.set_title('Spectral resolving power')
    #ax.plot(wave, np.ma.average(res_ma[:, 0, :-1], weights=aeff[:, :-1], axis=1), 'k:')
    ax.set_xlabel('wavelength [$\AA{}$]')
    ax.set_ylabel('resolving power')
    fig.subplots_adjust(bottom=0.13, left=0.13, top=.85)
    fig.savefig(os.path.join(get_path('figures'), 'respower.png'))
    fig.savefig(os.path.join(get_path('figures'), 'respower.pdf'))

Spectral resolving power for individual orders. Note that the upward spikes in resolving power are numerical artifacts:

- *gaps*: The resolving power is only calculated if a certain minimum number of photons is detected in an order. If most photons fall into a chip gap, then values on the curve might be missing.
- *spikes*: The resolving power is measured from the width of the detected photons distribution. If a spectral order falls very close to a chip gap, e.g. if half the photons remain undetected because they land between the CCDs, then the FWHM of the distribution of *detected photons* is only about 1/2 of the real value, leading to an upward spike in resolving power in the plot.

Since I understand where these features are coming from and it's easy to ignore them when looking by eye, I've decided to deal with them in detail later. For now, just ignore them.

In [ ]:
with mpl.style.context('fivethirtyeight'):
    plt.rc('axes', prop_cycle=(cycler('linestyle', ['-', '--', ':']) * 
                               cycler('color', ['r', 'b', 'y', 'c', 'g', 'm', '#7A68A6', '#188487', '#E24A33'])))

    fig = plt.figure()
    for i in apertures:
        ax = fig.add_subplot(2, 2, i + 1)
        ax = plot_res(res[:, i, :], ax)
        temp = ax.set_title('R for channel {}'.format(i))
        #ax.plot(wave, np.ma.average(res_ma[:, 0, :-1], weights=aeff[:, :-1], axis=1), 'k:')
        ax.set_xlabel('wavelength [$\AA{}$]')
        ax.set_ylabel('resolving power')
        
    fig.subplots_adjust(bottom=0.13, left=0.13, top=.85)
    #fig.savefig(os.path.join(get_path('figures'), 'respower.png'))
    #fig.savefig(os.path.join(get_path('figures'), 'respower.pdf'))

In [ ]:

with mpl.style.context('fivethirtyeight'):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(wave, np.ma.average(res_4[:, :-1], 
                                weights=np.ma.masked_equal(aeff_4[:, :-1], 0), axis=1),
           label='perfect alignment')
    ax.plot(mwave, np.ma.average(mres_4[:, :-1], 
                                 weights=np.ma.masked_equal(maeff_4[:, :-1], 0), axis=1),
           label='Baseline tolerance')
    ax.set_xlabel('wavelength [$\AA{}$]')
    ax.set_ylabel('resolving power')
    fig.subplots_adjust(bottom=0.13, left=0.13, top=.85)
    ax.legend()
    fig.savefig(os.path.join(get_path('figures'), 'averageres.png'))
    fig.savefig(os.path.join(get_path('figures'), 'averageres.pdf'))


This figure shows the resolving power averaged over all channels and all dispersed spectral orders. When averaging, values are weighted by their respective effective area. Some of the upward spikes seen in the previous plots are also visible here. In addition, there are some drops in $R$ that occur when a wavelength is observed where two orders with slightly different $R$ contribute and one of them falls into a chip gap for a certain wavelength. In the region $39-44\;\unicode[serif]{xC5}$ only a single order contributes, and so we see the $R$ rising with wavelength as in the previous plots. For small wavelengths $<12\unicode[serif]{xC5}$ very low orders contribute to the signal, bringing down the average $R$. 

However, over most of the wavelength range, $R$ is almost constant. That is not a coincidence, but rooted in the physics of CAT gratings. The spectral resolving power is measured as $R = \frac{d_x}{FWMH}$. The $FWHM$ is dominated by the mirror point spread function (PSF). In practice the PSF is slightly energy dependent, but this is not captured in the simple mirror model used for these simulations. All other effects on the $FWHM$ are very small (for example, it technically also depends on where on a CCD the signal falls since the CCDs are not curved and do not follow the Rowland circle exactly, but for Arcus this effect is negligible). So, the only variable left is $d_x$ the distance of the spot from the zeroth order. Photons of later wavelength are dispersed to larger distances and thus have larger $d_x$ which explains why $R$ increases with wavelength for every order. On the other hand, CAT gratings most efficiently diffract photons at a particular angle (the *blaze peak*). Thus, for longer and longer wavelengths the signal in an order becomes weaker, because the position of the order moves away from the blaze peak. At the same time, the next lower order moves towards the blaze peak and becomes increasingly stronger. In other words, the higher order (which has a higher $R$) drops in effective area, while the next lower order gains effective area. Together, this means that the averaged resolving power stays essentially constant.

## Figure of merit

In [ ]:
def combine_figures_of_merit(aeff, res):
    out = np.zeros(aeff.shape)
    for i in range(len(out)):
        res_i = res[:, 0, i]
        sort_res = np.argsort(res_i)
        merits = np.sqrt(res[:, i] * aeff[:, i])
        out[i] = np.nansum(merits[sort_res[:2]])

    return out

In [ ]:
def plot_merrit(aeff, res):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    #ax.plot(wave, combine_figures_of_merit(aeff, res), 'k',
    #            label='top 2 orders\nadded')
    for i in np.arange(3, 10):
        ax.plot(wave, np.sqrt(res[:, i] * aeff[:, i]),
                label='order {0}'.format(orders[i]))
    ax.legend()
    ax.set_xlabel('wavelength [$\AA{}$]')
    ax.set_ylabel('Figure of merrit [cm]')
    temp = ax.set_xlim([8, 55])
    return fig, ax

In [ ]:
fig, ax = plot_merrit(aeff_4, res_4)
ax.set_xlim([5, 50])
temp = ax.set_title('Figure of merit for line detection')

## An example on the detector
In the next few cells, I will run an example through the ray-trace code and make plots that show the actual detector image that this would produce.
Keep in mind that the current detector model is very simple: There is a uniform CCD contamination and an energy-dependend factor for the quantum efficiency, but otherwise the detector is perfect: No read-out streaks, no pile-up, and no chanrge-transfer inefficiency (all of which are to be expected in real detector) are simulated. Also, this particular simulation does not include any sort of background (astrophysical or instrumental).

For this example, I chose the spectrum of EQ Peg A (an active star) with the parameters from [Liefke et al. (2008)](http://adsabs.harvard.edu/abs/2008A%26A...491..859L). In practice, there is a close companion EQ Peg B and the two spectra will overlap in ARCUS, but for the purposes of this example, EQ Peg A alone will be good enough to represent an emission line spectrum.

The code below reads the input spectrum from table of energy and flux density that I generated with Sherpa (could have used XSPEC or ISIS, too).

In [ ]:
EQPegAspec = Table.read('../../inputdata/EQPegA_flux.tbl', format='ascii', names=['energy', 'flux'])
# restrict table to ARCUS energy range
EQPegAspec = EQPegAspec[(EQPegAspec['energy'] > 0.25) & (EQPegAspec['energy'] < 1.5)]

EQPegAcoord = astropy.coordinates.SkyCoord.from_name("EQ Peg A")

In [ ]:
plt.plot((EQPegAspec['energy'] * u.keV).to(u.Angstrom, equivalencies=u.spectral()), EQPegAspec['flux'])
plt.xlabel('wavelength [$\AA{}$]')
plt.ylabel('flux density [arbitrary scale]')
title = plt.title('EQ Peg A input spectrum')

In [ ]:
# Simulations runs for a while if we want a large number of photons and thus I run it in an external script.
# Read result for plotting here:
eqpegA = Table.read(os.path.join(get_path('rays'), 'EQPegA100ks.fits'))

In [ ]:
topccdids

In [ ]:
import matplotlib.colors

ind = eqpegA['CCD_ID'] >=0
ind = np.array([c in topccdids for c in eqpegA['CCD_ID']])
fig = plt.figure(figsize=(12,2))
ax = fig.add_subplot(111)
out = ax.hist2d(eqpegA['proj_x'][ind], eqpegA['proj_y'][ind], weights=eqpegA['probability'][ind],
          bins=[np.arange(-500,1200,10.), np.arange(-10, 10, .2)],
          )#norm=matplotlib.colors.LogNorm())
ax.set_xlabel('Focal plane coordinate [mm]')
temp = ax.set_ylabel('Focal plane coordinate [mm]')

Image of the total grating signal detected. The images is shown in focal plane coordinates, projected from the positions where the photons are detected on CCDs that follow the Rowland cirlce. This is **not** the same as the image a flat detector in the focal plane would see. **Note: X and y axis have very different scales!**

The spectrum seems to open up a little bit in the middle because the spectral focus does not coincide with the imaging focus. In other words, the detectors are placed to increase the spectral resolution, at the cost of the sub-optimal width in cross-dipersion direction.

In [ ]:
def rect_from_ccd_geometry(geometry):
    bl = geometry('center') + geometry('v_y') + geometry('v_z')
    width = -2. * geometry('v_y')[1]
    height = -2. * geometry('v_z')[2]
    # Scale that to fit in range [0, 1]
    return [(bl[1] + 200.) / 1200., bl[2]/1200.+0.5, width/1200., height/1200.]

In [ ]:
fig = plt.figure(figsize=(12,12))

for i in topccdids:
    rect = rect_from_ccd_geometry(instrum.det.elements[i].geometry)
    ax = fig.add_axes(rect)
    ind = eqpegA['CCD_ID'] == i
    hist, xedges, yedges = np.histogram2d(eqpegA['detpix_y'][ind], 2048-eqpegA['detpix_x'][ind], 
                            weights=eqpegA['probability'][ind],
                            range=[[0,1024], [0,2048]], bins=200
          )
    #mask out 0 as invalid
    hist[hist==0] = np.nan
    im = ax.imshow(hist, aspect='auto', interpolation='none', vmin=1., vmax=200)
    #ax.set_axis_off()
    ax.grid()
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    
#cax = plt.colorbar(im)
#fig.savefig(os.path.join(get_path('figures'), 'EQPegonCCD.pdf'), dpi=300)

Layout of the detectors as seen from the top. In 3-D space the detectors follow the Rowland circle, but they are plotted here as they would be seen from the top. Some dispersed photons fall beyond the boundaries of the chips shown here, but in order to restrict ARCUS to a managable number of chips, choices have to be made and this configuration maximizes the total signal for a fixed number of chips.

In [ ]:
fig = plt.figure(figsize=(12,6))

ax = fig.add_subplot(111)
ind = eqpegA['CCD_ID'] == 24
hist, xedges, yedges = np.histogram2d(eqpegA['detpix_y'][ind], eqpegA['detpix_x'][ind], 
                                weights=eqpegA['probability'][ind],
                            range=[[0,1024], [0,2048]], bins=[1024,2048]
          )
im = ax.imshow(hist, aspect='auto', interpolation='none', norm=matplotlib.colors.LogNorm())
ax.grid()
cax = plt.colorbar(im)
#ax.set_axis_off()

fig.savefig(os.path.join(get_path('figures'), 'EQPegonCCD24_log.pdf'), dpi=300)
fig.savefig(os.path.join(get_path('figures'), 'EQPegonCCD24_log.png'))

In [ ]:
fig = plt.figure(figsize=(12,12))
for j, i in enumerate(topccdids):
    ax = fig.add_subplot(8,2,j + 1)
    ind = eqpegA['CCD_ID'] == i
    hist, xedges, yedges = np.histogram2d(eqpegA['detpix_y'][ind], eqpegA['detpix_x'][ind], 
                                    weights=eqpegA['probability'][ind],
                                range=[[0,1024], [0,2048]], bins=[1024,2048]
              )
    ax.imshow(hist, aspect='auto', interpolation='none', norm=matplotlib.colors.LogNorm())
    ax.grid()

This plot show a detector image a single CCD. The image is binned to the size of the actual physical pixels and uses a logarithmic color scale. On the bottom is the spectal trace for one of the pairs of channels, at the top is the zeroth order for the other pair of channels.

## Order sorting
Different orders will overlap spatially on the CCD. We need to make use of the intrinsic energy resolution of the CCD to separate them. Below, I look at two different simulations (EQ Peg with lines and a contionuum source) to determine how good the CCDs have to be to fullfill this task.

In [ ]:
# Add a column with a "smeared CCD energy"
eqpegA['PI'] = eqpegA['energy'] * 1000 + np.random.randn(len(eqpegA)) * 65./2.35
ind = eqpegA['energy'] <= 1.
eqpegA['PI'][ind] = eqpegA['energy'][ind] * 1000 + np.random.randn(ind.sum()) * (5 + 8.6 * np.sqrt(3.2 * eqpegA['energy'][ind] + 16.))/2.35
# And, for plotting ad wavelenght column
eqpegA['wave'] = eqpegA['energy'].to(u.Angstrom, equivalencies=u.spectral()).value

In [ ]:
ind = (eqpegA['detpix_y'] < 500)

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)
out = ax.hist2d(eqpegA['proj_x'][ind], eqpegA['energy'][ind], weights=eqpegA['probability'][ind],
          bins=[np.arange(-50,1000,10.), np.arange(0, 1.5, .02)],
          )#norm=matplotlib.colors.LogNorm())
ax.set_ylabel('Energy [keV]')
ax.set_xlabel('focal plane coordiante [mm]')
temp = ax.set_title('Order sorting plot')

In [ ]:
dispersedangle = np.mean(eqpegA['detc1_phi'][ind & (eqpegA['order']==0)]) - eqpegA['detc1_phi']
mlambda = 2000. * np.sin(dispersedangle / 2)

ind = photonfiltertopccds(eqpegA) & (eqpegA['detpix_y'] < 500) & np.isfinite(eqpegA['order'])
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)
out = ax.hist2d(mlambda[ind],
                eqpegA['PI'][ind] / (mlambda[ind]*u.Angstrom).to(u.eV, equivalencies=u.spectral()), 
    
                weights=eqpegA['probability'][ind],
          bins=[np.arange(-1,  200., .1),np.arange(0, 14, .1)],
          norm=matplotlib.colors.LogNorm())
ax.set_xlabel('m $\lambda$ [$\AA$]')
ax.set_ylabel('PI  / (m $\lambda$)')
temp = ax.set_title('Order sorting plot')

In [ ]:
# For plotting add wavelength column
eqpegA['wave'] = eqpegA['energy'].to(u.Angstrom, equivalencies=u.spectral()).value

dispersedangle = np.mean(eqpegA['detc1_phi'][ind & (eqpegA['order']==0)]) - eqpegA['detc1_phi']
mlambda = 2000. * np.sin(dispersedangle / 2)
eqpegA['dispersedangle'] = dispersedangle
eqpegA['mlambda'] = mlambda

ind = photonfiltertopccds(eqpegA) & (eqpegA['detpix_y'] < 500) & np.isfinite(eqpegA['order'])


#Add a column with a "smeared CCD energy"
eqpegA['PI_Eric'] = eqpegA['energy'] * 1000 + np.random.randn(len(eqpegA)) * 65./2.35
ind1 = eqpegA['energy'] > 1.
eqpegA['PI_Eric'][ind1] = eqpegA['energy'][ind1] * 1000 + np.random.randn(ind1.sum()) * (5 + 8.6 * np.sqrt(3.2 * eqpegA['energy'][ind1] + 16.))/2.35

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(2,2,1)
out = np.histogram2d(mlambda[ind],
                     eqpegA['PI_Eric'][ind] / (mlambda[ind]*u.Angstrom).to(u.eV, equivalencies=u.spectral()), 
                     weights=eqpegA['probability'][ind],
                     bins=[np.arange(-1,  200., .1), np.arange(0, 14, .1)])
col = ax.imshow(out[0].T, vmax=50, origin='lower', aspect='auto', extent = (-1, 200, 0, 14))
ax.set_xlabel('m $\lambda$ [$\AA$]')
ax.set_ylabel('PI  / (m $\lambda$)')
ax.set_title("Eric's formula for CCD resolution")
plt.colorbar(col)

for i, s in enumerate([70., 80., 95.]):
    eqpegA['PI'] = eqpegA['energy'] * 1000 + np.random.randn(len(eqpegA)) * s/2.35
    out = np.histogram2d(mlambda[ind],
                    eqpegA['PI'][ind] / (mlambda[ind]*u.Angstrom).to(u.eV, equivalencies=u.spectral()), 
                    weights=eqpegA['probability'][ind],
                     bins=[np.arange(-1,  200., .1),np.arange(0, 14, .1)])

    ax = fig.add_subplot(2, 2, i + 2)
    col = ax.imshow(out[0].T, vmax=30, origin='lower', aspect='auto', extent = (-1, 200, 0, 14))
    ax.set_xlabel('m $\lambda$ [$\AA$]')
    ax.set_ylabel('PI  / (m $\lambda$)')
    ax.set_title("CCD FWHM = {0} eV".format(s))
    plt.colorbar(col)


In [ ]:
eqpegA_1 = eqpegA[ind]
eqpegA_1.write(os.path.join(get_path('rays'), 'EQPegA_ordersort.fits'), overwrite=True)

In [ ]:
flatspec = Table.read(os.path.join('/melkor/d1/guenther/processing/ARCUS/flatspecjitter000.fits'))
ind = photonfiltertopccds(flatspec) & (flatspec['detpix_y'] < 500) & np.isfinite(flatspec['order'])
# Note this is not EQ Peg, I just don't have time to rename everything
    
# For plotting add wavelength column
flatspec['wave'] = (flatspec['energy']).to(u.Angstrom, equivalencies=u.spectral()).value

dispersedangle = np.mean(flatspec['detc1_phi'][ind & (flatspec['order']==0)]) - flatspec['detc1_phi']
mlambda = 2000. * np.sin(dispersedangle / 2)
flatspec['dispersedangle'] = dispersedangle

#Add a column with a "smeared CCD energy"
flatspec['PI_Eric'] = flatspec['energy'] * 1000 + np.random.randn(len(flatspec)) * 65./2.35
ind1 = flatspec['energy'] > 1.
flatspec['PI_Eric'][ind1] = flatspec['energy'][ind1] * 1000 + np.random.randn(ind1.sum()) * (5 + 8.6 * np.sqrt(3.2 * flatspec['energy'][ind1] + 16.))/2.35

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(2,2,1)
out = np.histogram2d(mlambda[ind],
                     flatspec['PI_Eric'][ind] / (mlambda[ind]*u.Angstrom).to(u.eV, equivalencies=u.spectral()), 
                     weights=flatspec['probability'][ind],
                     bins=[np.arange(-1,  200., .4), np.arange(0, 14, .2)])
col = ax.imshow(out[0].T, vmax=10, origin='lower', aspect='auto', extent = (-1, 200, 0, 14))
ax.set_xlabel('m $\lambda$ [$\AA$]')
ax.set_ylabel('PI  / (m $\lambda$)')
ax.set_title("Suzaku-like CCDs")
plt.colorbar(col)

for i, s in enumerate([70., 80., 95.]):
    flatspec['PI'] = flatspec['energy'] * 1000 + np.random.randn(len(flatspec)) * s/2.35
    out = np.histogram2d(mlambda[ind],
                    flatspec['PI'][ind] / (mlambda[ind]*u.Angstrom).to(u.eV, equivalencies=u.spectral()), 
                    weights=flatspec['probability'][ind],
                     bins=[np.arange(-1,  200., .4),np.arange(0, 14, .2)])

    ax = fig.add_subplot(2, 2, i + 2)
    col = ax.imshow(out[0].T, vmax=10, origin='lower', aspect='auto', extent = (-1, 200, 0, 14))
    ax.set_xlabel('m $\lambda$ [$\AA$]')
    ax.set_ylabel('PI  / (m $\lambda$)')
    ax.set_title("CCD FWHM = {0} eV".format(s))
    plt.colorbar(col)

In [ ]:
# Similar plot for export
fig = plt.figure(figsize=(11,5))
for i, s in enumerate([65.,  95.]):
    flatspec['PI'] = flatspec['energy'] * 1000 + np.random.randn(len(flatspec)) * s/2.35
    out = np.histogram2d(mlambda[ind],
                    flatspec['PI'][ind] / (mlambda[ind]*u.Angstrom).to(u.eV, equivalencies=u.spectral()), 
                    weights=flatspec['probability'][ind],
                     bins=[np.arange(-1,  200., .4),np.arange(0, 14, .2)])

    ax = fig.add_subplot(1, 2, i + 1)
    col = ax.imshow(out[0].T, vmax=10, origin='lower', aspect='auto', extent = (-1, 200, 0, 14))
    ax.set_xlabel('m $\lambda$ [$\AA$]')
    ax.set_ylabel('PI  / (m $\lambda$)')
    ax.set_title("CCD FWHM = {0} eV".format(s))
    plt.colorbar(col)
    
fig.subplots_adjust(left=0.1, right=0.95)    
fig.savefig(os.path.join(get_path('figures'), 'OrderSorting.png'))
fig.savefig(os.path.join(get_path('figures'), 'OrderSorting.pdf'))

In [ ]:
display_codetoggle()